In [1]:
files = ['dumping-cleaned-news.txt', 'filtered-dumping-wiki.txt']

In [30]:
import random

pure_texts = []
for f in files:
    
    with open(f'/home/husein/pure-text/{f}') as fopen:
        txts = list(filter(None, fopen.read().split('\n')))

    pure_texts.extend(random.sample(txts, 200000))

In [10]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
import os

In [4]:
base_directory = '/home/husein/speech-bahasa'

In [5]:
df = pd.read_csv(f'{base_directory}/cv-corpus-5.1-2020-06-22/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(7490, 10)


In [6]:
id_commonvoice = []
for i in range(len(df)):
    p = f"{base_directory}/cv-corpus-5.1-2020-06-22/id/clips/{df['path'].iloc[i]}"
    t = df['sentence'].iloc[i]
    if len(t) < 5:
        continue
    id_commonvoice.append((p, t))

len(id_commonvoice)

7490

In [7]:
malay = glob(f'{base_directory}/part*/semisupervised/output-wav/*.wav')
len(malay)

377474

In [8]:
malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 4:
            continue
        malays.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 377474/377474 [01:11<00:00, 5300.51it/s]


In [11]:
wikipedia = []
wavs = glob(f'{base_directory}/streaming/*wav')
for i in tqdm(wavs):
    text = os.path.split(i)[1].replace('.wav', '')
    wikipedia.append((i, text))
    
len(wikipedia)

100%|██████████| 2887/2887 [00:00<00:00, 392765.35it/s]


2887

In [12]:
news = []
wavs = glob(f'{base_directory}/news/audio/*wav')

with open(f'{base_directory}/transcript-news.json') as fopen:
    transcript_news = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript_news[int(index)]
    news.append((i, text))

100%|██████████| 2151/2151 [00:00<00:00, 99362.85it/s]


In [13]:
audiobook = []
wavs = glob(f'{base_directory}/combined/*wav')
for i in tqdm(wavs):
    t = '/'.join(i.split('<>')[1:])
    t = t.split('.wav')[0]
    t = t.replace('output-wav', 'output-text')
    with open(f'{base_directory}/text-audiobook/{t}.wav.txt') as fopen:
        text = fopen.read()
    audiobook.append((i, text))

100%|██████████| 64341/64341 [00:05<00:00, 12620.71it/s]


In [14]:
df = pd.read_csv(f'{base_directory}/haqkiem/metadata.csv', header = None, sep = '|')
txts = df.values.tolist()
haqkiem = []
for f in tqdm(txts):
    text = f[1]
    text = text.split('.,,')[0]
    f = f[0]
    r = f'{base_directory}/haqkiem/{f}.wav'
    haqkiem.append((r, text))

100%|██████████| 4294/4294 [00:00<00:00, 783671.63it/s]


In [15]:
audios = id_commonvoice + malays + wikipedia + news + audiobook + haqkiem
audios, texts = zip(*audios)

In [34]:
import unicodedata
import re

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "'", "-", "z", "0", "1", "x", "2", "q", "*", "5", "3", "4", "6", "9", "8", "7", "%", "$", "\"", "/", "&", ":", "+"]
def preprocessing_text(string):
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c for c in string if c in vocabs])
    return re.sub(r'[ ]+', ' ', string).strip()

In [31]:
combined = pure_texts + list(texts)

In [32]:
random.shuffle(combined)

In [35]:
processed_text = [preprocessing_text(t) for t in tqdm(combined)]

 66%|██████▋   | 569356/858066 [00:13<00:06, 42572.14it/s]


100%|██████████| 858066/858066 [00:20<00:00, 42207.46it/s]A


In [36]:
with open('cleaned-rnn-lm.json', 'w') as fopen:
    json.dump(processed_text, fopen)